In [1]:
import re
import random
import pandas as pd
from docx import Document
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import genanki

def read_and_parse_questions(docx_path):
    document = Document(docx_path)
    text = '\n'.join([para.text for para in document.paragraphs])
    questions = re.split(r'\n\d+\.', text)
    questions = [q.strip() for q in questions if q.strip()]
    
    answer_key_index = questions[-1].find('Answer Key')
    answer_key = questions[-1][answer_key_index:]
    questions[-1] = questions[-1][:answer_key_index].strip()
    
    answers = re.findall(r'\d+\.\s*([A-D])', answer_key)
    
    parsed_questions = []
    for i, q in enumerate(questions):
        match = re.match(r'(.+?)\nA\)(.+?)\nB\)(.+?)\nC\)(.+?)\nD\)(.+)', q, re.DOTALL)
        if match:
            parsed_questions.append({
                'Question': match.group(1).strip(),
                'A': match.group(2).strip(),
                'B': match.group(3).strip(),
                'C': match.group(4).strip(),
                'D': match.group(5).strip(),
                'Correct Answer': answers[i]
            })
    return parsed_questions

def save_to_excel(questions, excel_path):
    df = pd.DataFrame(questions)
    df.to_excel(excel_path, index=False)

def preview_excel(excel_path):
    df = pd.read_excel(excel_path)
    print(df.head())
    if input("Is the Excel file correct? (y/n): ").lower() != 'y':
        print("Please correct the Excel file and run the script again.")
        exit()

def create_anki_deck_from_excel(excel_path, deck_name, output_path):
    df = pd.read_excel(excel_path)
    questions = df.to_dict('records')

    deck = genanki.Deck(random.randrange(1 << 30, 1 << 31), deck_name)
    model = genanki.Model(
        random.randrange(1 << 30, 1 << 31),
        'Simple Model',
        fields=[
            {'name': 'Question'},
            {'name': 'A'},
            {'name': 'B'},
            {'name': 'C'},
            {'name': 'D'},
            {'name': 'Correct Answer'},
        ],
        templates=[{
            'name': 'Card 1',
            'qfmt': '{{Question}}<br><br>A) {{A}}<br>B) {{B}}<br>C) {{C}}<br>D) {{D}}',
            'afmt': '{{Question}}<br><br>A) {{A}}<br>B) {{B}}<br>C) {{C}}<br>D) {{D}}<br><br><b>Correct Answer: {{Correct Answer}}</b>',
        }]
    )

    for q in questions:
        note = genanki.Note(
            model=model,
            fields=[q['Question'], q['A'], q['B'], q['C'], q['D'], q['Correct Answer']]
        )
        deck.add_note(note)
    
    genanki.Package(deck).write_to_file(output_path)

def main():
    Tk().withdraw()
    docx_path = askopenfilename(filetypes=[("Word Documents", "*.docx")])
    if not docx_path:
        print("No file selected.")
        return

    questions = read_and_parse_questions(docx_path)
    
    excel_path = asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel Files", "*.xlsx")])
    if not excel_path:
        print("No save location selected.")
        return

    save_to_excel(questions, excel_path)
    print(f"Questions have been saved to {excel_path}")
    
    preview_excel(excel_path)

    anki_path = asksaveasfilename(defaultextension=".apkg", filetypes=[("Anki Package", "*.apkg")])
    if not anki_path:
        print("No save location selected.")
        return

    deck_name = input("Enter the name of the Anki deck: ")
    create_anki_deck_from_excel(excel_path, deck_name, anki_path)
    print(f"Anki deck has been saved to {anki_path}")

if __name__ == "__main__":
    main()


No file selected.
